In [0]:
from pyspark.sql import SparkSession, functions as F
spark = SparkSession.builder.appName("Option_2_level_2").getOrCreate()

In [0]:
temp_read = spark.read.table("delta_silver_valid")

if not spark.catalog.tableExists("delta_facts_sales"):
    print("Table not exists")
    temp_read.write.format("delta").mode("overwrite").saveAsTable("delta_facts_sales")
    print("IF:Number of records in the table: ", temp_read.count())


else:
    temp_delta_facts_sales = spark.read.table("delta_facts_sales")
    max_timestamp = temp_delta_facts_sales.select(F.max("ingestion_timestamp")).collect()[0][0]

    temp_read = temp_read.filter(F.col("ingestion_timestamp") > max_timestamp)
    temp_read.write.format("delta").mode("append").saveAsTable("delta_facts_sales")

    print("Table exists")

    temp_check = spark.read.table("delta_facts_sales")
    print("else:Number of records in the table: ", temp_check.count())